In [1]:
from datetime import datetime, timedelta

In [2]:
import requests

In [3]:
import pandas as pd
import plotly.express as px

In [4]:
from IPython.display import clear_output

In [5]:
import re

### データ更新用（毎週末）

In [6]:
pos_data = pd.read_excel('cftc通貨先物ポジション_円.xlsx').iloc[:,1:]
pos_data.tail()
#pos_data.columns = ['date','long','short']
#pos_data.tail()

,date,commodity,long,short
13656,20240430,EMINI RUSSELL 1000 VALUE INDEX - CHICAGO MERCA...,2970,152
13657,20240430,MICRO E-MINI RUSSELL 2000 INDX - CHICAGO MERCA...,8705,17114
13658,20240430,NIKKEI STOCK AVERAGE - CHICAGO MERCANTILE EXCH...,1161,2952
13659,20240430,NIKKEI STOCK AVERAGE YEN DENOM - CHICAGO MERCA...,12994,4273
13660,20240430,S&P 500 ANNUAL DIVIDEND INDEX - CHICAGO MERCAN...,126760,16563


In [7]:
pos_data.dtypes

date          int64
commodity    object
long          int64
short         int64
dtype: object

In [8]:
today = datetime.today()

In [9]:
tuesdays = []
date = str(pos_data['date'].tolist()[-1])
date = datetime.strptime(date,'%Y%m%d')+ timedelta(days=7)
while True:
    if date > today:
        break
    else:
        # 残りの火曜日を見つける
        tuesdays.append(date.strftime('%Y%m%d'))
    date += timedelta(days=7)

In [10]:
tuesdays

['20240507',
 '20240514',
 '20240521',
 '20240528',
 '20240604',
 '20240611',
 '20240618',
 '20240625',
 '20240702',
 '20240709',
 '20240716',
 '20240723',
 '20240730',
 '20240806']

In [11]:
missed = []
i = 1
i = 1
for date in tuesdays:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        print('dd')
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/deacmesf"+date[-4:]+yyyy[-2:]+".htm"
    try:
        texts = requests.get(url).text
        pattern = r'<!--ih:includeHTML file="deacmesf.txt"-->([^"]+)<!--/ih:includeHTML-->'
        texts = re.findall(pattern, texts)[0]
        texts = texts.split('\r\n')
        texts = [s for s in texts if (s != '')&(s != ' ')]
        pos = [item for i, element in enumerate(texts) if ' - CHICAGO' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
        nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
        for s in nested_list:
            temp = [int(s) for s in s[9].replace(',','').split(' ') if not s == ''][:2]
            #print(temp)
            temp = pd.DataFrame([date,s[0]]+temp).T
            temp.columns=['date','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
    except:
        print(date)
        missed += [date]
        pass
    print(str(i)+'/'+str(len(tuesdays)))
    clear_output(True)
    i += 1
print('終わりました')

終わりました


In [12]:
pos_data.to_excel('cftc通貨先物ポジション_円.xlsx')

### データベースの構築用

In [6]:

def find_first_tuesday(year):
    # その年の1月1日
    date = datetime(year, 1, 1)
    # 1月1日が火曜日になるまで日付を進める
    while date.weekday() != 1:
        date += timedelta(days=1)
    return date

def find_tuesdays(start_year, end_year):
    tuesdays = []
    # 最初の火曜日を見つける
    date = find_first_tuesday(start_year)

    while True:
        if date.year == end_year + 1:
            break
        # 残りの火曜日を見つける
        tuesdays.append(date.strftime('%Y%m%d'))
        date += timedelta(days=7)  # 次の火曜日

    return tuesdays

In [7]:

# 関数を実行するための年を指定
start_year = 2018
end_year = 2024

start_year = 1998
end_year = 2009

# 結果を取得
tuesdays = find_tuesdays(start_year, end_year)

today = datetime.today().year

In [8]:
def find_yen(arg_list):
    key1 = 'JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE                           Code-097741'
    for i in range(len(arg_list)):
        text = arg_list[i]
        if text == key1:
            data = [s for s in arg_list[i+9].split(' ') if not s == '']
            return [int(data[0].replace(',','')),int(data[1].replace(',','')),int(data[0].replace(',','')) - int(data[1].replace(',',''))]
        else:
            pass

In [9]:
#tuesdays = ['20240220','20240227']

https://www.cftc.gov/sites/default/files/files/dea/cotarchives/2022/futures/financial_lf052422.htm

In [10]:
a = r'fg'+r'fa'
a

'fgfa'

In [11]:
pos_data = pd.DataFrame(columns=['date','commodity','long','short'])
missed = []
i = 1
for date in tuesdays:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        print('future')
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/deacmesf"+date[-4:]+yyyy[-2:]+".htm"
    try:
        texts = requests.get(url).text
        pattern = r'<!--ih:includeHTML file="deacmesf.txt"-->([^"]+)<!--/ih:includeHTML-->'
        texts = re.findall(pattern, texts)[0]
        texts = texts.split('\r\n')
        texts = [s for s in texts if (s != '')&(s != ' ')]
        if date == '20220517':
            texts[0] ='USD Malaysian Crude Palm Oil C - CHICAGO MERCANTILE EXCHANGE         Code-037021'

        pos = [item for i, element in enumerate(texts) if ' - CHICAGO' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
        nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
        for s in nested_list:
            temp = [int(s) for s in s[9].replace(',','').split(' ') if not s == ''][:2]
            #print(temp)
            temp = pd.DataFrame([date,s[0]]+temp).T
            temp.columns=['date','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
    except:
        print(date)
        missed += [date]
        pass
    print(str(i)+'/'+str(len(tuesdays)))
    clear_output(True)
    i += 1
print('終わりました')

終わりました


In [ ]:
pos_data

In [ ]:
missed

In [ ]:
adj = ['20181224','20181231','20191230','20201221','20230703']

In [ ]:
missed2 = []
i = 1
for date in adj:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/deacmesf"+date[-4:]+yyyy[-2:]+".htm"
    try:
        texts = requests.get(url).text
        pattern = r'<!--ih:includeHTML file="deacmesf.txt"-->([^"]+)<!--/ih:includeHTML-->'
        texts = re.findall(pattern, texts)[0]
        texts = texts.split('\r\n')
        texts = [s for s in texts if (s != '')&(s != ' ')]
        pos = [item for i, element in enumerate(texts) if ' - CHICAGO' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
        nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
        for s in nested_list:
            temp = [int(s) for s in s[9].replace(',','').split(' ') if not s == ''][:2]
            #print(temp)
            temp = pd.DataFrame([date,s[0]]+temp).T
            temp.columns=['date','commodity','long','short']
            pos_data = pd.concat([pos_data,temp],axis=0)
    except:
        print(date)
        missed2 += [date]
        pass
    print(str(i)+'/'+str(len(adj)))
    clear_output(True)
    i += 1
print('終わりました')

In [ ]:
pos_data = pos_data.sort_values('date')

In [ ]:
pos_data.to_excel('cftc通貨先物ポジション_円_IT_GFC.xlsx')

In [ ]:
missed2 = []
i = 1
for date in ['20220517','20220524']:
    if datetime.today() < datetime.strptime(date,'%Y%m%d'):
        break
    yyyy = date[:4]
    url = "https://www.cftc.gov/sites/default/files/files/dea/cotarchives/"
    url += yyyy+"/futures/deacmesf"+date[-4:]+yyyy[-2:]+".htm"
    #try:
    texts = requests.get(url).text
    pattern = r'<!--ih:includeHTML file="deacmesf.txt"-->([^"]+)<!--/ih:includeHTML-->'
    texts = re.findall(pattern, texts)[0]
    texts = texts.split('\r\n')
    texts = [s for s in texts if (s != '')&(s != ' ')]
    if date == '20220517':
        texts[0] ='USD Malaysian Crude Palm Oil C - CHICAGO MERCANTILE EXCHANGE         Code-037021'
    pos = [item for i, element in enumerate(texts) if ' - CHICAGO' in element for item in ((i-1, i) if i > 0 else (i,))]
        #print(pos)
    nested_list = [texts[pos[i]:pos[i+1]] for i in range(0,len(pos)-1,2)]
        #[print(s[9]) for s in nested_list]
    for s in nested_list:
        try:
            temp = [int(s) for s in s[9].replace(',','').split(' ') if not s == ''][:2]
        except:
            print(s)
            break
            #print(temp)
        temp = pd.DataFrame([date,s[0]]+temp).T
        temp.columns=['date','commodity','long','short']
        pos_data = pd.concat([pos_data,temp],axis=0)
    #except:
    #    print(date)
    #    missed2 += [date]
    #    pass
    print(str(i)+'/'+str(len(tuesdays)))
    #clear_output(True)
    i += 1
print('終わりました')

In [ ]:
missed2

In [ ]:
pos_data.to_excel('cftc通貨先物ポジション_円.xlsx')

In [ ]:
import re
# 文字列を定義
prefix = r"Hello"
suffix = r"World"

# 連結して正規表現パターンを作成
pattern = prefix + r".*" + suffix

# テストする文字列
test_string = "Hello, how are you? World"

# 正規表現を使って検索
match = re.search(pattern, test_string)

if match:
    print("一致が見つかりました。")
else:
    print("一致が見つかりませんでした。")